In [220]:
import requests
from time import sleep
import json
import time
import pandas as pd
import numpy as np
from datetime import date
from selenium import webdriver
from bs4 import BeautifulSoup
import re

In [218]:
#from 'cleaning-for-BA'
GranCat = pd.read_csv("LowLevelCatagoryRegex.csv", header=None, skiprows=[0])

def process_cat(df):
    # get all the values in category regex
    subCats = GranCat.iloc[:, 1:]
    Cats = []
    for index, row in subCats.iterrows():
        for i in GranCat.columns[1:]:
            if pd.isnull(row[i]): continue
            else: Cats.append(row[i])
               
    # assign potential category for each clothing row based on title, else other
    clothesCat = []
    for index, row in df.iterrows():
        rowCat = float("NaN")
        for cat in Cats:
            if re.search(cat, row['display_name'].lower()):
                rowCat = cat
                print(rowCat)
                break
        if pd.isnull(rowCat):
            clothesCat.append('other')
        else:
            clothesCat.append(rowCat)

    return clothesCat

# set parent category to each product
def get_cat(x):
    if x == 'other': return x
    else: return [GranCat[0][i] for i in GranCat.index if x in GranCat.iloc[i].to_list()][0]

# get low_level column given a df
def get_lowlevel(df):
    df['low_level_new'] = process_cat(df)
    df['low_level_new'] = df['low_level_new'].apply(lambda x: get_cat(x))
    return df
# get unique id
def get_id(df, date='20210901'):
    brand = df['brand_name'][0]
    clothing_ids = []
    for i in range(len(df.index)):
        clothing_id = date + brand + str(i)
        clothing_ids.append(clothing_id)
    return clothing_ids
def get_dic_image(row):
    color_image_dict = {'color': 'image_url'}
    for i in range(len(row['color'])):
        #print(i)
        #{row['color'][i]: row['image'][i]}
        try:
            color_image_dict = {row['color'][i]:row['image'][i]}
        except:
            color_image_dict = {row['color'][i]:row['image'][0]}
            

        
    return color_image_dict

def get_price(x): 
    try: return x.split(': ')[1]
    except: return 'Unknown'

def change_gender(x):
    if "Men" in x and "women" not in x.lower(): return 'Men'
    else: return 'Women'

In [456]:
driver = webdriver.Chrome(executable_path='/Users/carrieoh/Desktop/Fall 2021/Changing Room/chromedriver')

In [445]:
ef_url_clothing = 'https://www.eileenfisher.com/clothing'

In [4]:
driver.get(ef_url_clothing)

In [12]:
ef = pd.DataFrame(columns=['display_name', 'product_material', 'color', 'size', 'price', 'product_url', 'brand_name', 'description', 'scrapped_date', 'low_level', 'gender', 'secondhand'])


In [13]:
for page in range(1, 31, 1):
    
    page_url = "https://www.eileenfisher.com/clothing?p=" + str(page)
    driver.get(page_url)
    
    name = driver.find_elements_by_class_name("product-item-link") #.text
    img_url = driver.find_elements_by_class_name('src_updated') #get_attribute('src')
    
    for i in range(len(name)):
        ef = ef.append({'display_name': name[i].text,
                        'product_url': img_url[i].get_attribute('src'), 
                        'brand_name': 'Eileen Fisher',
                        'scrapped_date':date.today(),
                        'secondhand':'No'}, ignore_index=True)    

#closing the driver
#driver.close()

In [16]:
links = []
for page in range(1, 31, 1):
    
    page_url = "https://www.eileenfisher.com/clothing?p=" + str(page)
    driver.get(page_url)
    
    price = driver.find_elements_by_class_name("price") #.text #remove first 11 for all pages
    
    j = (page-1)*24
    for i in range(11,len(price)):
        ef['price'][j] = price[i].text
        j += 1
    
    item_link = driver.find_elements_by_css_selector("div.product-images-container > a")
    
    for i in range(len(item_link)):
        links.append(item_link[i].get_attribute('href'))

In [431]:
ef.head(1)

,display_name,product_material,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,gender,secondhand,low_level_new
0,TRACEABLE ORGANIC COTTON JERSEY SHORTS,90% Organic Cotton/10% Spandex,[BLACK],"[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$108.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 1/2-inch elasti...",2021-10-21,Shorts,Women,No,shorts


In [18]:
ef.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   display_name      709 non-null    object
 1   product_material  0 non-null      object
 2   color             0 non-null      object
 3   size              0 non-null      object
 4   price             709 non-null    object
 5   product_url       709 non-null    object
 6   brand_name        709 non-null    object
 7   description       0 non-null      object
 8   scrapped_date     709 non-null    object
 9   low_level         0 non-null      object
 10  gender            0 non-null      object
 11  secondhand        709 non-null    object
dtypes: object(12)
memory usage: 66.6+ KB


In [19]:
assert len(links) == len(ef)

In [430]:
ef.head(1)

,display_name,product_material,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,gender,secondhand,low_level_new
0,TRACEABLE ORGANIC COTTON JERSEY SHORTS,90% Organic Cotton/10% Spandex,[BLACK],"[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$108.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 1/2-inch elasti...",2021-10-21,Shorts,Women,No,shorts


In [122]:
#Description
count = 0
for l in links:
    driver.get(l)
    soup = BeautifulSoup(driver.page_source, "lxml")
    description = ''
    for d in soup.find('div',{'class':'short-list-attribute'}).find_all('li'):
        description += d.get_text() + " "
    ef['description'][count] = description
    count += 1

In [470]:
#rest of the attributes
count = 383
for l in links[383:]:
    driver.get(l)
    soup = BeautifulSoup(driver.page_source, "lxml")
    
    #Product_material
    try:
        material = str(soup.find_all('div',{'class':'short-list-attribute'})[1].find('li')).split('.')[1].strip()
    except: 
        pass
    
    #Color + image_link_color
    img_link_c = {}
    colors = []
    c_ids = []
    color = soup.find('div',{'attribute-code':'color'}).find_all('img')
    c_link = soup.find_all('div',{'attribute-code':'color'})[0].find('div').find_all('div')
    for i in range(len(color)):
        colors.append(color[i].get('alt'))
    time.sleep(3)
    for i in range(len(c_link)):
        c_ids.append(c_link[i].get('id'))
        try:
            for i in range(len(c_ids)): 
                driver.find_element_by_xpath('//*[@id="'+str(c_ids[i])+'"]').click()
                img_link_c[colors[i]] = driver.find_element_by_class_name('s7flyoutzoom').find_element_by_css_selector('div > img:nth-child(1)').get_attribute('src')
        except:
            pass
    #print(count, img_link_c)
    
    #Size
    sizes = []
    try:
        size = soup.find('div',{'aria-labelledby':'option-label-size-173'}).find_all('span')
        for i in range(len(size)):
            sizes.append(size[i].text)
    except:
        pass
    
    ef['product_material'][count] = material
    ef['color'][count] = colors 
    ef['size'][count] = sizes
    ef['image_link_color'][count] = img_link_c
    count += 1
   

383 {'BARLEY': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EF48486-243?$M2_FLYOUT_ZOOM$&id=XtBlP3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'GRAPHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EF48486-243?$M2_FLYOUT_ZOOM$&id=XtBlP3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
384 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EF45410D-001?$M2_FLYOUT_ZOOM$&id=rhgle2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'MOON': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EF45410D-087?$M2_FLYOUT_ZOOM$&id=AHmlo2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'IVORY': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EF45410D-101?$M2_FLYOUT_ZOOM$&id=tQllh2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
385 {'BLUSH': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F0TTU-P4272M-681?$M2_FLYOUT_ZOOM$&id=L2vlu3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher

399 {}
400 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EF47900A-770?$M2_FLYOUT_ZOOM$&id=EgHlU3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'YARROW': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EF47900A-770?$M2_FLYOUT_ZOOM$&id=EgHlU3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
401 {'MAPLE OAT': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1QRU-K5196M-264?$M2_FLYOUT_ZOOM$&id=-TvlO3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'MOON': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1QRU-K5196M-264?$M2_FLYOUT_ZOOM$&id=-TvlO3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
402 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1GDD-P8542M-001?$M2_FLYOUT_ZOOM$&id=mzmll3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'LIGHT BARLEY': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1GDD-P8542M-129?$M2_FLYOUT_ZOOM$&id=c0Pll0&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
403 {'BLACK': 'https://s7ondemand

426 {'BLACK/SOFT WHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1NDI-T5704M-012?$M2_FLYOUT_ZOOM$&id=es4ma2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
427 {'GRAPHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/E2SOQ-T4368M-142?$M2_FLYOUT_ZOOM$&id=jUnlU2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'DARK PEARL': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/E2SOQ-T4368M-142?$M2_FLYOUT_ZOOM$&id=jUnlU2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
428 {'HONEY': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/R0OVC-A2148M-703?$M2_FLYOUT_ZOOM$&id=wmAlL2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/R0OVC-A2148M-703?$M2_FLYOUT_ZOOM$&id=wmAlL2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
429 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1VF-D4952M-001?$M2_FLYOUT_ZOOM$&id=mTEl01&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
430 {'MAPLE OAT': 'h

461 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EEVFF-T0011M-100?$M2_FLYOUT_ZOOM$&id=sIJlc1&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555', 'WHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EEVFF-T0011M-100?$M2_FLYOUT_ZOOM$&id=sIJlc1&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
462 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1TLL-P4586M-250?$M2_FLYOUT_ZOOM$&id=oeklU1&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555', 'KHAKI': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1TLL-P4586M-250?$M2_FLYOUT_ZOOM$&id=oeklU1&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
463 {'WHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EETK-P0696M-100?$M2_FLYOUT_ZOOM$&id=aiGlM0&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555', 'BONE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EETK-P0696M-100?$M2_FLYOUT_ZOOM$&id=aiGlM0&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
464 {'BLACK': 'https://s7ondemand5.scene7.com/

494 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1FGF-K5207M-001?$M2_FLYOUT_ZOOM$&id=_HGl70&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555', 'WHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1FGF-K5207M-100?$M2_FLYOUT_ZOOM$&id=andl42&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555', 'BLACKBERRY': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1FGF-K5207M-559?$M2_FLYOUT_ZOOM$&id=7HqlQ0&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
495 {'ASH': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1HOE-T5685M-030?$M2_FLYOUT_ZOOM$&id=Zgtlx2&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
496 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1TK-P4566M-001?$M2_FLYOUT_ZOOM$&id=J_nll1&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
497 {'YOUNG FERN': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1GAF-A0835M-448?$M2_FLYOUT_ZOOM$&id=K3rlM3&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555', 'DELPHINE': 'https://s7ondemand

520 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1OKB-D4978M-001?$M2_FLYOUT_ZOOM$&id=KwylB2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
521 {'SEAWEED': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1RHD-K5184M-349?$M2_FLYOUT_ZOOM$&id=7SKlJ1&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1RHD-K5184M-349?$M2_FLYOUT_ZOOM$&id=7SKlJ1&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
522 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1FTJ-T5768M-001?$M2_FLYOUT_ZOOM$&id=ZKzlQ2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'BLACKBERRY': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1FTJ-T5768M-001?$M2_FLYOUT_ZOOM$&id=ZKzlQ2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
523 {'WHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/EEFTJ-U4217M-100?$M2_FLYOUT_ZOOM$&id=_QRla2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
524 {'SEAWEED': 'https://s7onde

558 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1FTJ-P4597M-001?$M2_FLYOUT_ZOOM$&id=nzklK3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'SLATE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1FTJ-P4597M-001?$M2_FLYOUT_ZOOM$&id=nzklK3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
559 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1OJ1-T5754M-554?$M2_FLYOUT_ZOOM$&id=RHtli2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'CASSIS': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1OJ1-T5754M-554?$M2_FLYOUT_ZOOM$&id=RHtli2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
560 {'CHALK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1JQM-P4566M-113?$M2_FLYOUT_ZOOM$&id=hcHlb0&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1JQM-P4566M-001?$M2_FLYOUT_ZOOM$&id=QMZl61&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
561 {'BLACK': 'https://s7ondemand5.scene7.

592 {'SEAWEED': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1SUZ-P8284M-349?$M2_FLYOUT_ZOOM$&id=CDQl71&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1SUZ-P8284M-001?$M2_FLYOUT_ZOOM$&id=68Hlx1&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'WHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1SUZ-P8284M-100?$M2_FLYOUT_ZOOM$&id=QrnlT3&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'CLAY': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1SUZ-P8284M-221?$M2_FLYOUT_ZOOM$&id=sqGlK0&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'KHAKI': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1SUZ-P8284M-250?$M2_FLYOUT_ZOOM$&id=fcnlY2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'OLIVE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1SUZ-P8284M-301?$M2_FLYOUT_ZOOM$&id=TlClA2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
593 {'BLACK': 'https://s7ondemand5.scene7.com/is/ima

615 {'SEAWEED': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1AJT-T5260M-349?$M2_FLYOUT_ZOOM$&id=shJlc1&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555', 'WHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1AJT-T5260M-349?$M2_FLYOUT_ZOOM$&id=shJlc1&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
616 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1TJF-P4366M-001?$M2_FLYOUT_ZOOM$&id=X9vlZ1&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
617 {}
618 {'CASSIS': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/R0TK-P0696M-554?$M2_FLYOUT_ZOOM$&id=nZJlu1&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
619 {}
620 {}
621 {}
622 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1TJF-P4313M-001?$M2_FLYOUT_ZOOM$&id=yPqly0&fmt=jpg&dpr=off&fit=constrain,1&wid=416&hei=555'}
623 {}
624 {}
625 {'CHARCOAL': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/F1QWD-W5901M-686?$M2_FLYOUT_ZOOM$&id=x36l02&fmt=jpg&dpr=off&fit=constrain,1&wid=

664 {'SEAWEED': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1MNZ-K5182M-349?$M2_FLYOUT_ZOOM$&id=UNrlZ2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1MNZ-K5182M-001?$M2_FLYOUT_ZOOM$&id=PDsls2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'WHITE': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1MNZ-K5182M-100?$M2_FLYOUT_ZOOM$&id=VQglX0&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
665 {'BLACK': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1OJ1-D4991M-001?$M2_FLYOUT_ZOOM$&id=3qSly1&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'AQUA': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1OJ1-D4991M-468?$M2_FLYOUT_ZOOM$&id=oTYl20&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544', 'AEGEAN': 'https://s7ondemand5.scene7.com/is/image/EileenFisher/S1OJ1-D4991M-488?$M2_FLYOUT_ZOOM$&id=RoRlQ2&fmt=jpg&dpr=off&fit=constrain,1&wid=408&hei=544'}
666 {}
667 {}
668 {}
669 {'BLACK': 'https://s7

In [471]:
ef

,display_name,product_material,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,gender,secondhand,low_level_new
0,TRACEABLE ORGANIC COTTON JERSEY SHORTS,90% Organic Cotton/10% Spandex,[BLACK],"[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$108.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 1/2-inch elasti...",2021-10-21,Shorts,Women,No,shorts
1,ITALIAN CASHMERE SCRUNCH NECK TOP,100% Cashmere,"[BLACK, CHARCOAL, BARLEY, YOUNG FERN, POWDER, ...","[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$358.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Black only available online. Barley, Corn Silk...",2021-10-21,T-Shirts,Women,No,tops?
2,VELVET DROP-WAIST DRESS,80% Viscose/20% Silk,[BLACK],"[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$348.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Midi Dresses,Women,No,dresse?s?$
3,SILK GEORGETTE CREPE PLEATED PANT,100% Silk,[BLACK],"[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$258.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Casual Pants,Women,No,pants?
4,SILK GEORGETTE CREPE V-NECK SHIRT,100% Silk,"[BLACK, DARK CRANBERRY]","[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$278.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model in Dark Cranberry is 5'9"" and wearing a ...",2021-10-21,Long Sleeve Buttondowns,Women,No,shirts?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,SYSTEM LIGHTWEIGHT WASHABLE STRETCH CREPE TAPE...,66% Viscose/27% Nylon/7% Spandex,[BLACK],"[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$188.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1-inch elastic waistb...",2021-10-21,Casual Pants,Women,No,pants?
705,SYSTEM LIGHTWEIGHT WASHABLE STRETCH CREPE WIDE...,66% Viscose/27% Nylon/7% Spandex,[BLACK],"[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$198.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 1/2-inch elastic wa...",2021-10-21,Casual Pants,Women,No,pants?
706,SYSTEM WASHABLE STRETCH CREPE SLIM ANKLE PANT,</li>,"[BLACK, WHITE, BONE]","[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$158.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 3/4-inch smooth ela...",2021-10-21,Casual Pants,Women,No,pants?
707,SYSTEM ULTRAFINE MERINO LONG CARDIGAN IN RESPO...,100% Superfine Merino Wool,"[BROWNSTONE, BLACK]","[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$188.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{},Eileen Fisher,"Model is 5'10"" and wearing a size small, which...",2021-10-21,Cardigans,Women,No,cardigan


In [472]:
ef.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   display_name      709 non-null    object
 1   product_material  709 non-null    object
 2   color             709 non-null    object
 3   size              709 non-null    object
 4   price             709 non-null    object
 5   product_url       709 non-null    object
 6   image_link_color  709 non-null    object
 7   brand_name        709 non-null    object
 8   description       709 non-null    object
 9   scrapped_date     709 non-null    object
 10  low_level         709 non-null    object
 11  gender            709 non-null    object
 12  secondhand        709 non-null    object
 13  low_level_new     709 non-null    object
dtypes: object(14)
memory usage: 77.7+ KB


In [550]:
pattern = "\d{1,3}[$%]"
materials = {}
m = ef['product_material'][0].split('/')
for item in m:
    percent = re.findall(pattern,item)[0]
    mat = re.findall('[a-zA-Z_]+',item)
    material = ''
    for i in range(len(mat)):
        if i < len(mat)-1:
            material += mat[i] + " "
        else:
            material += mat[i]
    materials[material] = percent
materials

{'Organic Cotton': '90%', 'Spandex': '10%'}

In [593]:
#clean up materials
ef['new_product_material'] = ''
for i in range(len(ef['product_material'])):
    if ef['product_material'][i] == '</li>':
        pass
    else:
        pattern = "\d{1,3}[$%]"
        m = ef['product_material'][i].split('/')
        materials = {}
        for item in m:
            try:
                percent = re.findall(pattern,item)[0]
                mat = re.findall('[a-zA-Z_]+',item)
                material = ''
                for j in range(len(mat)):
                    if j < len(mat)-1:
                        material += mat[j] + " "
                    else:
                        material += mat[j]
            except:
                pass
            materials[material] = percent
        ef['new_product_material'][i] = materials


In [595]:
#ef['product_material'] = ef['new_product_material']
#ef = ef.drop(columns=['new_product_material'])

In [600]:
ef

,display_name,product_material,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,gender,secondhand
0,TRACEABLE ORGANIC COTTON JERSEY SHORTS,"{'Organic Cotton': '90%', 'Spandex': '10%'}",[BLACK],"[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$108.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 1/2-inch elasti...",2021-10-21,Shorts,Women,No
1,ITALIAN CASHMERE SCRUNCH NECK TOP,{'Cashmere': '100%'},"[BLACK, CHARCOAL, BARLEY, YOUNG FERN, POWDER, ...","[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$358.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Black only available online. Barley, Corn Silk...",2021-10-21,T-Shirts,Women,No
2,VELVET DROP-WAIST DRESS,"{'Viscose': '80%', 'Silk': '20%'}",[BLACK],"[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$348.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Midi Dresses,Women,No
3,SILK GEORGETTE CREPE PLEATED PANT,{'Silk': '100%'},[BLACK],"[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$258.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Casual Pants,Women,No
4,SILK GEORGETTE CREPE V-NECK SHIRT,{'Silk': '100%'},"[BLACK, DARK CRANBERRY]","[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$278.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model in Dark Cranberry is 5'9"" and wearing a ...",2021-10-21,Long Sleeve Buttondowns,Women,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,SYSTEM LIGHTWEIGHT WASHABLE STRETCH CREPE TAPE...,"{'Viscose': '66%', 'Nylon': '27%', 'Spandex': ...",[BLACK],"[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$188.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1-inch elastic waistb...",2021-10-21,Casual Pants,Women,No
705,SYSTEM LIGHTWEIGHT WASHABLE STRETCH CREPE WIDE...,"{'Viscose': '66%', 'Nylon': '27%', 'Spandex': ...",[BLACK],"[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$198.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 1/2-inch elastic wa...",2021-10-21,Casual Pants,Women,No
706,SYSTEM WASHABLE STRETCH CREPE SLIM ANKLE PANT,,"[BLACK, WHITE, BONE]","[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$158.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 3/4-inch smooth ela...",2021-10-21,Casual Pants,Women,No
707,SYSTEM ULTRAFINE MERINO LONG CARDIGAN IN RESPO...,{'Superfine Merino Wool': '100%'},"[BROWNSTONE, BLACK]","[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$188.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{},Eileen Fisher,"Model is 5'10"" and wearing a size small, which...",2021-10-21,Cardigans,Women,No


In [476]:
#ef['low_level'] = ef['low_level_new']
#ef = ef.drop(columns=['low_level_new'])

In [229]:
#gender
ef['gender'] = 'Women'

In [477]:
ef.head(5)

,display_name,product_material,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,gender,secondhand
0,TRACEABLE ORGANIC COTTON JERSEY SHORTS,90% Organic Cotton/10% Spandex,[BLACK],"[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$108.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 1/2-inch elasti...",2021-10-21,Shorts,Women,No
1,ITALIAN CASHMERE SCRUNCH NECK TOP,100% Cashmere,"[BLACK, CHARCOAL, BARLEY, YOUNG FERN, POWDER, ...","[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$358.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Black only available online. Barley, Corn Silk...",2021-10-21,T-Shirts,Women,No
2,VELVET DROP-WAIST DRESS,80% Viscose/20% Silk,[BLACK],"[PP, PS, PM, PL, XXS, XS, S, M, L, XL, 1X, 2X,...",$348.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Midi Dresses,Women,No
3,SILK GEORGETTE CREPE PLEATED PANT,100% Silk,[BLACK],"[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$258.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Casual Pants,Women,No
4,SILK GEORGETTE CREPE V-NECK SHIRT,100% Silk,"[BLACK, DARK CRANBERRY]","[XXS, XS, S, M, L, XL, 1X, 2X, 3X]",$278.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model in Dark Cranberry is 5'9"" and wearing a ...",2021-10-21,Long Sleeve Buttondowns,Women,No


In [601]:
ef_csv = ef.to_csv('eileenfisher_table.csv', index=False)

In [602]:
w = pd.read_csv('eileenfisher_table.csv')
w.head(20)

,display_name,product_material,color,size,price,product_url,image_link_color,brand_name,description,scrapped_date,low_level,gender,secondhand
0,TRACEABLE ORGANIC COTTON JERSEY SHORTS,"{'Organic Cotton': '90%', 'Spandex': '10%'}",['BLACK'],"['XXS', 'XS', 'S', 'M', 'L', 'XL', '1X', '2X',...",$108.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model featured is 5'10"". 1 1/2-inch elasti...",2021-10-21,Shorts,Women,No
1,ITALIAN CASHMERE SCRUNCH NECK TOP,{'Cashmere': '100%'},"['BLACK', 'CHARCOAL', 'BARLEY', 'YOUNG FERN', ...","['PP', 'PS', 'PM', 'PL', 'XXS', 'XS', 'S', 'M'...",$358.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Black only available online. Barley, Corn Silk...",2021-10-21,T-Shirts,Women,No
2,VELVET DROP-WAIST DRESS,"{'Viscose': '80%', 'Silk': '20%'}",['BLACK'],"['PP', 'PS', 'PM', 'PL', 'XXS', 'XS', 'S', 'M'...",$348.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Midi Dresses,Women,No
3,SILK GEORGETTE CREPE PLEATED PANT,{'Silk': '100%'},['BLACK'],"['XXS', 'XS', 'S', 'M', 'L', 'XL', '1X', '2X',...",$258.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Casual Pants,Women,No
4,SILK GEORGETTE CREPE V-NECK SHIRT,{'Silk': '100%'},"['BLACK', 'DARK CRANBERRY']","['XXS', 'XS', 'S', 'M', 'L', 'XL', '1X', '2X',...",$278.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,"Model in Dark Cranberry is 5'9"" and wearing a ...",2021-10-21,Long Sleeve Buttondowns,Women,No
5,SILK GEORGETTE CREPE MOCK NECK TANK,{'Silk': '100%'},['BLACK'],"['XXS', 'XS', 'S', 'M', 'L', 'XL', '1X', '2X',...",$218.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Tank Tops,Women,No
6,EGGSHELL RECYCLED NYLON HOODED LONG COAT,{'Recycled Nylon': '100%'},['BLACK'],"['XXS', 'XS', 'S', 'M', 'L', 'XL', '1X', '2X',...",$398.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Only available at select EILEEN FISHER stores....,2021-10-21,Coats,Women,No
7,RECYCLED CASHMERE WOOL CARDIGAN,NaN,"['ASH', 'AEGEAN TEAL']","['PP', 'PS', 'PM', 'PL', 'XXS', 'XS', 'S', 'M'...",$448.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'ASH': 'https://s7ondemand5.scene7.com/is/ima...,Eileen Fisher,"Aegean Teal only available online. 5'9"" model ...",2021-10-21,Cardigans,Women,No
8,MERINO CREW NECK TOP IN RESPONSIBLE WOOL,{'Extra Fine Merino Wool': '100%'},"['CHARCOAL', 'CRANBERRY', 'SEAFOAM']","['XXS', 'XS', 'S', 'M', 'L', 'XL', '1X', '2X',...",$228.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'CHARCOAL': 'https://s7ondemand5.scene7.com/i...,Eileen Fisher,"Model in Cranberry is 5'10"" and wearing a size...",2021-10-21,T-Shirts,Women,No
9,EGGSHELL RECYCLED NYLON HOODED COAT,{'Recycled Nylon': '100%'},"['BLACK', 'POWDER', 'IVY']","['PP', 'PS', 'PM', 'PL', 'XXS', 'XS', 'S', 'M'...",$358.00,https://s7ondemand5.scene7.com/is/image/Eileen...,{'BLACK': 'https://s7ondemand5.scene7.com/is/i...,Eileen Fisher,Select sizes and colors may vary by store. Pet...,2021-10-21,Coats,Women,No
